The given csv's contain lists of doctors, their corresponding calls, transactions, and engagements. The task is to combine them into one sheet that displays when doctor calls and transactions occurred relative to the marketing campaign engagements. The result should display each month in which a doctor was engaged and how many calls and transactions happened before and after that engagement to determine how successful the engagement was.

In [1]:
import pandas as pd
calls_n_trxs = pd.read_csv("IQ2020.csv")
engagements = pd.read_csv("2021-ULD-ENGAGE.csv")

Dropping the prescriber's last name for privacy in sharing this code

In [2]:
calls_n_trxs = calls_n_trxs.drop(columns=["Prescriber Last Name"])

Cleaning up calls_n_trxs and engagements

In [3]:
calls_n_trxs = calls_n_trxs.fillna(0)

In [4]:
engagements = engagements.fillna(0)

In [5]:
calls_n_trxs = calls_n_trxs.drop([0, 1, 8608])

In [6]:
engagements = engagements.drop(4239)

In [7]:
engagements = engagements[engagements["Total"] > 0]

NPI is the Prescriber's ID 

In [8]:
engagements["NPI"]

0       1.003075e+09
1       1.003080e+09
2       1.003107e+09
3       1.003209e+09
4       1.003222e+09
            ...     
1572    1.992706e+09
1573    1.992734e+09
1574    1.992772e+09
1575    1.992869e+09
1576    1.992887e+09
Name: NPI, Length: 1522, dtype: float64

In [9]:
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

To merge the two effectively, we must prepare both engagements and calls_n_trxs. Expanding engagements by month and casting the NPIs to floats in calls_n_trxs

In [10]:
engagements_long = pd.melt(engagements, id_vars=['NPI'], value_vars=months)

In [11]:
engagements_long = engagements_long[engagements_long["value"] > 0]

In [12]:
engagements_long

,NPI,variable,value
0,1.003075e+09,Jan,1.0
3,1.003209e+09,Jan,1.0
4,1.003222e+09,Jan,1.0
5,1.003235e+09,Jan,2.0
6,1.003303e+09,Jan,2.0
...,...,...,...
18240,1.972917e+09,Dec,3.0
18249,1.982671e+09,Dec,1.0
18257,1.982957e+09,Dec,2.0
18258,1.992281e+09,Dec,3.0


In [13]:
calls_n_trxs["Prescriber NPI"] = calls_n_trxs["Prescriber NPI"].astype(float)

In [14]:
All = pd.merge(engagements_long, calls_n_trxs, how='inner', left_on="NPI", right_on="Prescriber NPI")

In [15]:
calls_n_trxs["Prescriber NPI"].unique()[0]

1538170774.0

Confirming we have the correct amount of NPIs

In [16]:
engagements["NPI"].unique()

array([1.00307511e+09, 1.00307992e+09, 1.00310668e+09, ...,
       1.99277198e+09, 1.99286944e+09, 1.99288748e+09])

In [17]:
len(set.intersection(set(calls_n_trxs["Prescriber NPI"].unique()), set(engagements["NPI"].unique())))

1183

In [18]:
total = 0

for i in engagements["NPI"].unique():
    if i in calls_n_trxs["Prescriber NPI"].unique():
        total += 1
        
print(total)

1183


In [19]:
All

,NPI,variable,value,Territory Name ID,Prescriber ID,Prescriber NPI,Prescriber First Name,12/01/2019\nTRx,12/01/2019\nCalls Count,01/01/2020\nTRx,...,08/01/2021\nTRx,08/01/2021\nCalls Count,09/01/2021\nTRx,09/01/2021\nCalls Count,10/01/2021\nTRx,10/01/2021\nCalls Count,11/01/2021\nTRx,11/01/2021\nCalls Count,12/01/2021\nTRx,12/01/2021\nCalls Count
0,1.003209e+09,Jan,1.0,SA01020507 - St Louis MO,6855569,1.003209e+09,JEREMY,0.0,0.0,0.0,...,0.0,1,0.0,1,0.0,1,0.0,1,0.0,0
1,1.003209e+09,Feb,2.0,SA01020507 - St Louis MO,6855569,1.003209e+09,JEREMY,0.0,0.0,0.0,...,0.0,1,0.0,1,0.0,1,0.0,1,0.0,0
2,1.003209e+09,Mar,2.0,SA01020507 - St Louis MO,6855569,1.003209e+09,JEREMY,0.0,0.0,0.0,...,0.0,1,0.0,1,0.0,1,0.0,1,0.0,0
3,1.003209e+09,Apr,2.0,SA01020507 - St Louis MO,6855569,1.003209e+09,JEREMY,0.0,0.0,0.0,...,0.0,1,0.0,1,0.0,1,0.0,1,0.0,0
4,1.003209e+09,May,2.0,SA01020507 - St Louis MO,6855569,1.003209e+09,JEREMY,0.0,0.0,0.0,...,0.0,1,0.0,1,0.0,1,0.0,1,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5418,1.861405e+09,Dec,1.0,SA01029902 - WS South,0819888,1.861405e+09,DELANEY,0.0,0.0,0.0,...,0.0,0,0.0,0,0.0,1,0.0,0,0.0,1
5419,1.871554e+09,Dec,1.0,SA01029903 - WS West,2396360,1.871554e+09,TRACY,0.0,0.0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,2
5420,1.932483e+09,Dec,1.0,SA01020408 - Dallas TX,6393244,1.932483e+09,ROYA,0.0,0.0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,1
5421,1.942473e+09,Dec,1.0,SA01029903 - WS West,4421422,1.942473e+09,PRADNYA,0.0,0.0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,1


In [20]:
pd.set_option("display.max_columns", None)

In [21]:
columns = ["12/01/2019\nTRx", "12/01/2019\nCalls Count", "01/01/2020\nTRx", "01/01/2020\nCalls Count", "02/01/2020\nTRx", "02/01/2020\nCalls Count", "03/01/2020\nTRx", "03/01/2020\nCalls Count", "04/01/2020\nTRx", "04/01/2020\nCalls Count", "05/01/2020\nTRx", "05/01/2020\nCalls Count", "06/01/2020\nTRx", "06/01/2020\nCalls Count", "07/01/2020\nTRx", "07/01/2020\nCalls Count", "08/01/2020\nTRx", "08/01/2020\nCalls Count", "09/01/2020\nTRx", "09/01/2020\nCalls Count", "10/01/2020\nTRx", "10/01/2020\nCalls Count", "11/01/2020\nTRx", "11/01/2020\nCalls Count", "12/01/2020\nTRx", "12/01/2020\nCalls Count", "01/01/2021\nTRx", "01/01/2021\nCalls Count", "02/01/2021\nTRx", "02/01/2021\nCalls Count", "03/01/2021\nTRx", "03/01/2021\nCalls Count", "04/01/2021\nTRx", "04/01/2021\nCalls Count", "05/01/2021\nTRx", "05/01/2021\nCalls Count", "06/01/2021\nTRx", "06/01/2021\nCalls Count", "07/01/2021\nTRx", "07/01/2021\nCalls Count", "08/01/2021\nTRx", "08/01/2021\nCalls Count", "09/01/2021\nTRx", "09/01/2021\nCalls Count", "10/01/2021\nTRx", "10/01/2021\nCalls Count", "11/01/2021\nTRx", "11/01/2021\nCalls Count", "12/01/2021\nTRx", "12/01/2021\nCalls Count"]

In [22]:
months_to_nums = {
    "Jan" : "01",
    "Feb" : "02",
    "Mar" : "03",
    "Apr" : "04",
    "May" : "05",
    "Jun" : "06",
    "Jul" : "07",
    "Aug" : "08",
    "Sep" : "09",
    "Oct" : "10",
    "Nov" : "11",
    "Dec" : "12"
}

This is the core of the program that calculates how many calls and transactions happened before and after each engagement on a monthly basis.

In [23]:
before_calls_ = []
before_trx_ = []
after_calls_ = []
after_trx_ = []

for row in All.index:
    month = All["variable"][row]
    month_num = months_to_nums[month]
    year_num = "2021"
    correct_col = month_num + "/01/" + year_num + "\nTRx"
    before_columns = columns[:columns.index(correct_col)]
    after_columns = columns[columns.index(correct_col):]
    
    before_calls = 0
    before_trx = 0
    

    for column in before_columns:
        #print(column)
        value = int(All[column][row])
        if column[-3:] == "TRx":
            before_trx += value
            #print("Adding " + str(value) + " to TRx before")
        else:
            before_calls += value
            #print("Adding " + str(value) + " to Calls before")
            
    before_calls_.append(before_calls)
    before_trx_.append(before_trx)
    
    after_calls = 0
    after_trx = 0
    
    for column in after_columns:
        value = int(All[column][row])
        if column[-3:] == "TRx":
            after_trx += value
            #print("Adding " + str(value) + " to TRx after")
        else:
            after_calls += value
            #print("Adding " + str(value) + " to Calls after")
    
    after_calls_.append(after_calls)
    after_trx_.append(after_trx)
    

All["Before Calls"] = before_calls_
All["Before TRx"] = before_trx_
All["After Calls"] = after_calls_
All["After TRx"] = after_trx_

In [24]:
All = All.rename(columns={"variable": "Engagement Month", "value": "Amount of Engagements"})

In [25]:
All

,NPI,Engagement Month,Amount of Engagements,Territory Name ID,Prescriber ID,Prescriber NPI,Prescriber First Name,12/01/2019\nTRx,12/01/2019\nCalls Count,01/01/2020\nTRx,01/01/2020\nCalls Count,02/01/2020\nTRx,02/01/2020\nCalls Count,03/01/2020\nTRx,03/01/2020\nCalls Count,04/01/2020\nTRx,04/01/2020\nCalls Count,05/01/2020\nTRx,05/01/2020\nCalls Count,06/01/2020\nTRx,06/01/2020\nCalls Count,07/01/2020\nTRx,07/01/2020\nCalls Count,08/01/2020\nTRx,08/01/2020\nCalls Count,09/01/2020\nTRx,09/01/2020\nCalls Count,10/01/2020\nTRx,10/01/2020\nCalls Count,11/01/2020\nTRx,11/01/2020\nCalls Count,12/01/2020\nTRx,12/01/2020\nCalls Count,01/01/2021\nTRx,01/01/2021\nCalls Count,02/01/2021\nTRx,02/01/2021\nCalls Count,03/01/2021\nTRx,03/01/2021\nCalls Count,04/01/2021\nTRx,04/01/2021\nCalls Count,05/01/2021\nTRx,05/01/2021\nCalls Count,06/01/2021\nTRx,06/01/2021\nCalls Count,07/01/2021\nTRx,07/01/2021\nCalls Count,08/01/2021\nTRx,08/01/2021\nCalls Count,09/01/2021\nTRx,09/01/2021\nCalls Count,10/01/2021\nTRx,10/01/2021\nCalls Count,11/01/2021\nTRx,11/01/2021\nCalls Count,12/01/2021\nTRx,12/01/2021\nCalls Count,Before Calls,Before TRx,After Calls,After TRx
0,1.003209e+09,Jan,1.0,SA01020507 - St Louis MO,6855569,1.003209e+09,JEREMY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,2,0.0,1,0.0,2,0.0,2,0.0,2,0.0,2,0.0,1,0.0,1,0.0,1,0.0,1,0.0,1,0.0,1,0.0,0,2,0,15,0
1,1.003209e+09,Feb,2.0,SA01020507 - St Louis MO,6855569,1.003209e+09,JEREMY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,2,0.0,1,0.0,2,0.0,2,0.0,2,0.0,2,0.0,1,0.0,1,0.0,1,0.0,1,0.0,1,0.0,1,0.0,0,3,0,14,0
2,1.003209e+09,Mar,2.0,SA01020507 - St Louis MO,6855569,1.003209e+09,JEREMY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,2,0.0,1,0.0,2,0.0,2,0.0,2,0.0,2,0.0,1,0.0,1,0.0,1,0.0,1,0.0,1,0.0,1,0.0,0,5,0,12,0
3,1.003209e+09,Apr,2.0,SA01020507 - St Louis MO,6855569,1.003209e+09,JEREMY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,2,0.0,1,0.0,2,0.0,2,0.0,2,0.0,2,0.0,1,0.0,1,0.0,1,0.0,1,0.0,1,0.0,1,0.0,0,7,0,10,0
4,1.003209e+09,May,2.0,SA01020507 - St Louis MO,6855569,1.003209e+09,JEREMY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,2,0.0,1,0.0,2,0.0,2,0.0,2,0.0,2,0.0,1,0.0,1,0.0,1,0.0,1,0.0,1,0.0,1,0.0,0,9,0,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5418,1.861405e+09,Dec,1.0,SA01029902 - WS South,0819888,1.861405e+09,DELANEY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,1,0.0,0,0.0,1,1,0,1,0
5419,1.871554e+09,Dec,1.0,SA01029903 - WS West,2396360,1.871554e+09,TRACY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,2,0,0,2,0
5420,1.932483e+09,Dec,1.0,SA01020408 - Dallas TX,6393244,1.932483e+09,ROYA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,1,0,0,1,0
5421,1.942473e+09,Dec,1.0,SA01029903 - WS West,4421422,1.942473e+09,PRADNYA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,1,0,0,1,0


In [26]:
AllwoColumns = All.drop(columns=columns)

In [27]:
AllwoColumns.to_csv("Jan23Deliverable.csv")